In [15]:
import pandas as pd 
import numpy as np
from fangraphs_projections import get_projections
import pre_process_script as pp


In [16]:
league = pd.read_csv('./data/league/league_current.csv')

In [17]:
# create function to update projections
def update_projections(projections_1, projections_2):
    # if any projections are missing, retunr the other set of projections
    if projections_1 is None:
        return projections_2
    elif projections_2 is None:
        return projections_1
    
    # if projections are not missing, average them
    stats = [c for c in projections_1.columns if c not in ["PlayerName", "minpos", "ADP", "Team", "ShortName"]]
    new_projections = (projections_1[stats] + projections_2[stats]) 
    # new_projections["PlayerName"] = projections_1["PlayerName"]
    # new_projections[["PlayerName"]] = projections_1[["PlayerName"]]
    # Reorder columns
    # new_projections = new_projections.reindex(columns=["PlayerName"] + stats)
    
    return new_projections


In [18]:
b = get_projections("all", "bat", "atc").set_index("playerid")
b[b.PlayerName == "CJ Abrams"].T

playerid,25768
PlayerName,CJ Abrams
minpos,SS
ADP,222.759995
Team,WSN
ShortName,Nationals
G,137
AB,494
PA,530
H,126
1B,90


In [19]:
names = ["steamer", "   ", "zipsdc", "atc", "thebat", "zips"]
av_names = []   
i = 0
for n in ["atc"]:
    print(n)
    # try:
    # Get projections from Fangraphs
    if i == 0:
        bats  = get_projections("all", "bat", n).set_index("playerid")
        pitch = get_projections("all", "pit", n).set_index("playerid")
    else:
        bats  = update_projections(bats, get_projections("all", "bat", n).set_index("playerid"))
        pitch = update_projections(pitch, get_projections("all", "pit", n).set_index("playerid"))
    # ranks = pitch.add(bats, fill_value=0).sort_values("POINTS")
    # player_list = player_list.join(ranks, how="inner")
    # player_list.rename(columns={"POINTS":"POINTS_{}".format(n)}, inplace=True)
    # av_names.append("POINTS_{}".format(n))
    # except:
    #     print("{} data unavailable.".format(n))
    i += 1
# Drop players with mising data
# bats.dropna(inplace=True)
# pitch.dropna(inplace=True)
# Average projections   
bats = bats #/ len(names)
pitch = pitch #/ len(names)   


# Rename colums so pitchers hace sufix _P and batters have sufix _B if columns are not in ["PlayerName", "minpos", "ADP", "Team", "ShortName"]
bat_cols = [c for c in bats.columns if c not in ["PlayerName", "minpos", "ADP", "Team", "ShortName"]]
pitch_cols = [c for c in pitch.columns if c not in ["PlayerName", "minpos", "ADP", "Team", "ShortName"]]
# Rename columns
bats.rename(columns={c:c + "_B" for c in bat_cols}, inplace=True)
pitch.rename(columns={c:c + "_P" for c in pitch_cols}, inplace=True)
# bats.columns = [c + "_B" for c in bats.columns]
# pitch.columns = [c + "_P" for c in pitch.columns]

# Reset index
bats.reset_index(inplace=True)
pitch.reset_index(inplace=True)

atc


In [20]:
# Combine projections
projections = pd.concat([bats, pitch]).fillna(0)

In [21]:
projections

,playerid,PlayerName,minpos,ADP,Team,ShortName,G_B,AB_B,PA_B,H_B,...,BS_P,IP_P,TBF_P,H_P,R_P,ER_P,HR_P,SO_P,BB_P,HBP_P
0,15640,Aaron Judge,OF,3.200000,NYY,Yankees,149.0,545.0,645.0,155.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20123,Juan Soto,OF,10.790000,SDP,Padres,148.0,508.0,647.0,143.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10155,Mike Trout,OF,19.030001,LAA,Angels,139.0,510.0,602.0,137.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,23697,Julio Rodríguez,OF,4.220000,SEA,Mariners,146.0,568.0,634.0,158.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,13510,José Ramírez,3B,3.670000,CLE,Guardians,151.0,570.0,650.0,156.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
830,2608,Jhoulys Chacín,0,999.000000,0,0,0.0,0.0,0.0,0.0,...,0,29.0,129.0,31.0,18.0,16.0,4.0,24.0,12.0,1.0
831,6986,Ian Kennedy,0,742.010010,TEX,Rangers,0.0,0.0,0.0,0.0,...,0,30.0,130.0,32.0,18.0,16.0,6.0,28.0,10.0,1.0
832,25862,Noah Davis,0,999.000000,COL,Rockies,0.0,0.0,0.0,0.0,...,0,21.0,93.0,23.0,14.0,13.0,4.0,19.0,9.0,1.0
833,10855,Chasen Shreve,0,999.000000,DET,Tigers,0.0,0.0,0.0,0.0,...,0,35.0,150.0,33.0,19.0,17.0,5.0,32.0,15.0,1.0


In [53]:
# Load league data
id_dict = pd.read_csv('/Users/mitchv34/Work/fantasy_baseball_tools/data/SFBB Player ID Map - PLAYERIDMAP.csv')
id_dict = dict(zip( id_dict.FANTRAXID.to_list(), id_dict.IDFANGRAPHS.to_list() ))

# player_list = pd.read_csv("data/league/Fantrax-Players-Believers in BABIP (BIB).csv") # Old rosters
player_list = pd.read_csv("data/league/league_current.csv") # New rosters
player_list.ID = player_list.ID.apply(lambda x: id_dict[x] if x in id_dict.keys() else np.nan)

# Rename ID to playerid
player_list.rename(columns={"ID": "playerid"}, inplace=True)

In [54]:
# Taken Players
## Drop FA players 
player_list_taken = player_list[player_list.Status != "FA"]
## Drop waivers players
player_list_taken = player_list_taken[player_list_taken.Status.str.contains("W ") == False]

In [55]:
player_list_taken.Status.value_counts()

uu          42
Pete        41
FAFO        40
SS268       40
Tomy        40
TCE         40
DN          39
TYR         39
Patron#5    39
CR          39
sfernand    39
Edgar       36
Name: Status, dtype: int64

In [38]:
# Merge projections with player list
player_list = player_list[["playerid", "Player", "Team", "Position", "Status"]].merge(projections, how="inner", on="playerid")

In [48]:
team_stats = player_list_taken.groupby("Status").sum()

In [49]:
team_stats.columns

Index(['ADP', 'G_B', 'AB_B', 'PA_B', 'H_B', '1B_B', '2B_B', '3B_B', 'HR_B',
       'R_B', 'RBI_B', 'BB_B', 'IBB_B', 'SO_B', 'HBP_B', 'SF_B', 'SH_B',
       'SB_B', 'CS_B', 'GS_P', 'G_P', 'SV_P', 'HLD_P', 'BS_P', 'IP_P', 'TBF_P',
       'H_P', 'R_P', 'ER_P', 'HR_P', 'SO_P', 'BB_P', 'HBP_P'],
      dtype='object')

In [51]:
# Batting Stats
team_stats["AVG"] =  team_stats["H_B"] / team_stats["AB_B"]
team_stats["OBP"] = (team_stats["H_B"] + team_stats["BB_B"] + team_stats["HBP_B"]) / (team_stats["PA_B"])
team_stats["SLG"] = (team_stats["H_B"] + team_stats["2B_B"] + 2*team_stats["3B_B"] + 3*team_stats["HR_B"]) / team_stats["AB_B"]
team_stats["OPS"] = team_stats["OBP"] + team_stats["SLG"]
team_stats["NSB"] = team_stats["SB_B"] - team_stats["CS_B"]
# Pitching Stats
team_stats["ERA"] = (team_stats["ER_P"] / team_stats["IP_P"]) * 9
team_stats["WHIP"] = (team_stats["H_P"] + team_stats["BB_P"]) / team_stats["IP_P"]
team_stats["K/9"] = team_stats["SO_P"] / team_stats["IP_P"] * 9
team_stats["BB/9"] = team_stats["BB_P"] / team_stats["IP_P"] * 9
team_stats["K/BB"] = team_stats["SO_P"] / team_stats["BB_P"]
team_stats["H/IP"] = team_stats["H_P"] / team_stats["IP_P"]
team_stats["NSV"] = team_stats["SV_P"] - team_stats["BS_P"]
team_stats["NSVH"] = (team_stats["SV_P"]  + team_stats["HLD_P"] - team_stats["BS_P"])
team_stats["RA"] = team_stats["G_P"] - team_stats["GS_P"]

def get_QS(df):
    QS = np.round(  + 0.173737*df['IP_P'] +  0.003827*df['SO_P'] -0.074804*df['BB_P'] -0.122394*df['HR_P'] -0.055386*(df['H_P']-df['HR_P']) )
    return QS

team_stats["QS"] = get_QS(team_stats)

In [52]:
# Create empty dataframe to store rankings with team names as index
team_rankings = pd.DataFrame(index=team_stats.index)

# Batting Stats
team_rankings[["H", "R", "HR", "RBI", "BB", "NSB", "AVG", "OBP", "SLG", "OPS"]] = team_stats[["H_B", "R_B", "HR_B", "RBI_B", "BB_B", "NSB", "AVG", "OBP", "SLG", "OPS"]].rank(ascending=True) - 1
team_rankings["K"] = team_stats["SO_B"].rank(ascending=False) - 1

# Pitching Stats
team_rankings[["IP", "RA", "QS", "K/9","K/BB", "NSV", "NSVH"]] = team_stats[["IP_P", "RA", "QS", "K/9","K/BB", "NSV", "NSVH"]].rank(ascending=True) - 1
team_rankings[["ERA", "WHIP", "BB/9", "H/IP"]] = team_stats[["ERA", "WHIP", "BB/9", "H/IP"]].rank(ascending=False) - 1

# Add total points
team_rankings["Total"] = team_rankings.sum(axis=1)
# Set "Totla" as first column
cols = team_rankings.columns.tolist()
cols = cols[-1:] + cols[:-1]
team_rankings = team_rankings[cols]
# Sort by total points
team_rankings.sort_values("Total", ascending=False, inplace=True)

# #Convert all columns to int
# team_rankings = team_rankings.astype(int) 

# # Reset index
# team_rankings.reset_index(inplace=True)
# # Rename index column to team
# team_rankings.rename(columns={"Status": "Team"}, inplace=True)
# # Set index starting at 1
# team_rankings.index += 1


team_rankings

,Total,H,R,HR,RBI,BB,NSB,AVG,OBP,SLG,...,RA,QS,K/9,K/BB,NSV,NSVH,ERA,WHIP,BB/9,H/IP
Status,,,,,,,,,,,,,,,,,,,,,
uu,179.0,7.0,7.0,11.0,9.0,9.0,10.0,5.0,8.0,11.0,...,7.0,11.0,9.0,4.0,11.0,10.0,7.0,6.0,3.0,9.0
FAFO,134.0,6.0,5.0,6.0,4.0,3.0,3.0,11.0,10.0,10.0,...,5.0,9.0,7.0,2.0,9.0,5.0,3.0,2.0,1.0,5.0
TYR,132.0,1.0,2.0,2.0,0.0,5.0,5.0,7.0,9.0,7.0,...,9.0,8.0,6.0,6.0,8.0,7.0,5.0,7.0,7.0,6.0
sfernand,131.5,8.0,8.0,4.5,6.0,11.0,6.0,8.0,11.0,1.0,...,3.0,5.0,4.0,11.0,0.0,0.0,6.0,8.0,11.0,3.0
CR,128.0,3.0,4.0,7.0,5.0,7.0,1.0,1.0,6.0,8.0,...,2.0,6.0,8.0,10.0,4.0,3.0,11.0,10.0,9.0,8.0
Pete,126.0,4.0,6.0,8.0,8.0,8.0,0.0,0.0,2.0,4.0,...,4.0,4.0,10.0,8.0,7.0,8.0,10.0,11.0,6.0,11.0
DN,123.5,2.0,0.5,3.0,3.0,6.0,4.0,2.0,5.0,6.0,...,6.0,1.0,11.0,9.0,10.0,9.0,8.0,9.0,5.0,10.0
Edgar,116.5,11.0,11.0,10.0,11.0,10.0,11.0,6.0,4.0,5.0,...,0.0,0.0,3.0,5.0,2.5,2.0,9.0,4.0,4.0,4.0
TCE,111.5,0.0,0.5,4.5,2.0,1.0,2.0,4.0,7.0,9.0,...,11.0,7.0,2.0,7.0,2.5,11.0,2.0,5.0,10.0,2.0


In [ ]:
team_rankings.loc[1]

Team      uu
Total    190
H          6
R          7
HR        11
RBI        8
BB         9
NSB        9
AVG        5
OBP        9
SLG       11
OPS       11
K          4
IP        11
RA        10
QS        11
K/9       10
K/BB       5
NSV       11
NSVH      11
ERA       10
WHIP       7
BB/9       3
H/IP      11
Name: 1, dtype: object

In [ ]:
team_stats

,ADP,G_B,AB_B,PA_B,H_B,1B_B,2B_B,3B_B,HR_B,R_B,...,ERA,WHIP,K/9,BB/9,K/BB,H/IP,NSV,NSVH,RA,QS
Status,,,,,,,,,,,,,,,,,,,,,
CR,3732.280003,1872.0,6942.0,7875.0,1770.0,1071.0,367.0,30.0,305.0,1017.0,...,3.311387,1.118513,10.317583,2.523625,4.088398,0.838110,89.0,115.0,275.0,134.0
Champs,5327.950010,1409.0,5199.0,5891.0,1369.0,850.0,268.0,19.0,236.0,774.0,...,3.607843,1.172840,9.385621,2.692810,3.485437,0.873638,70.0,122.0,329.0,137.0
DN,3161.460001,1601.0,5905.0,6696.0,1524.0,936.0,308.0,26.0,255.0,860.0,...,3.609553,1.151630,10.057619,2.736164,3.675810,0.847612,91.0,116.0,251.0,132.0
Edgar,4494.579986,1854.0,6908.0,7770.0,1832.0,1170.0,364.0,31.0,265.0,1018.0,...,3.621600,1.204000,9.100800,2.772000,3.283117,0.896000,22.0,73.0,188.0,122.0
Freeman,4449.260006,1648.0,6141.0,6940.0,1642.0,1042.0,316.0,21.0,264.0,938.0,...,3.620192,1.214744,9.699519,3.274038,2.962555,0.850962,72.0,105.0,275.0,120.0
La Maqui,4193.979939,1831.0,6925.0,7688.0,1771.0,1092.0,367.0,27.0,286.0,993.0,...,3.680541,1.198751,9.571280,3.006243,3.183801,0.864724,52.0,72.0,193.0,93.0
Patron#5,2657.600036,1529.0,5773.0,6413.0,1519.0,960.0,303.0,31.0,223.0,832.0,...,3.623377,1.205937,9.100186,2.871985,3.168605,0.886827,56.0,78.0,191.0,105.0
Pete,4256.530006,1717.0,6320.0,7174.0,1586.0,968.0,312.0,18.0,289.0,906.0,...,3.518852,1.143443,9.833607,2.692623,3.652055,0.844262,72.0,112.0,263.0,123.0
TCE,6751.619938,1483.0,5523.0,6254.0,1426.0,865.0,282.0,24.0,253.0,830.0,...,3.755396,1.195683,9.297842,2.725899,3.410926,0.892806,45.0,120.0,367.0,136.0


In [ ]:
player_list[player_list.Status == "Champs"]

,playerid,Player,Team_x,Position,Status,PlayerName,minpos,ADP,Team_y,ShortName,...,BS_P,IP_P,TBF_P,H_P,R_P,ER_P,HR_P,SO_P,BB_P,HBP_P
2,13510,Jose Ramirez,CLE,"3B,INF",Champs,José Ramírez,3B,3.530000,CLE,Guardians,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16149,Aaron Nola,PHI,SP,Champs,Aaron Nola,0,33.360001,PHI,Phillies,...,0,194.0,779.0,167.0,83.0,76.0,23.0,217.0,40.0,9.0
10,26410,Alek Manoah,TOR,SP,Champs,Alek Manoah,0,68.089996,TOR,Blue Jays,...,0,190.0,784.0,162.0,84.0,77.0,23.0,183.0,55.0,14.0
20,18000,Triston McKenzie,CLE,SP,Champs,Triston McKenzie,0,85.150002,CLE,Guardians,...,0,180.0,733.0,151.0,84.0,78.0,27.0,185.0,54.0,6.0
31,25377,Brady Singer,KC,SP,Champs,Brady Singer,0,178.820007,KCR,Royals,...,0,179.0,753.0,171.0,84.0,77.0,20.0,166.0,52.0,11.0
63,21032,Emmanuel Clase,CLE,RP,Champs,Emmanuel Clase,0,27.910000,CLE,Guardians,...,0,65.0,255.0,51.0,18.0,17.0,4.0,68.0,14.0,2.0
66,6562,Alex Cobb,SF,SP,Champs,Alex Cobb,0,220.649994,SFG,Giants,...,0,155.0,649.0,147.0,66.0,61.0,11.0,151.0,46.0,5.0
68,13624,Corey Seager,TEX,"SS,INF",Champs,Corey Seager,SS,57.880001,TEX,Rangers,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,19556,Yordan Alvarez,HOU,OF,Champs,Yordan Alvarez,DH/OF,11.270000,HOU,Astros,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,19470,Tommy Edman,STL,"2B,SS,INF",Champs,Tommy Edman,2B,74.860001,STL,Cardinals,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Count the number of "SP" on each team
player_list[player_list.Position.apply(lambda x : ("SP" in x) & ("RP" not in x))].groupby("Status").count()

,playerid,Player,Team_x,Position,PlayerName,minpos,ADP,Team_y,ShortName,G_B,...,BS_P,IP_P,TBF_P,H_P,R_P,ER_P,HR_P,SO_P,BB_P,HBP_P
Status,,,,,,,,,,,,,,,,,,,,,
CR,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
Champs,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
DN,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
Edgar,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
FA,107,107,107,107,107,107,107,107,107,107,...,107,107,107,107,107,107,107,107,107,107
Freeman,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
La Maqui,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
Patron#5,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
Pete,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
